# Classification of Nigerian Traditional Attire
### Step 1: Collection and Organization of the dataset

### Step 2: Spliting the dataset into training and validation set

In [1]:
# Import Libraries and Set Device
# Import essential libraries
import torch
import torch.nn as nn
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader
import time

# Select device: GPU if available, otherwise CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device


device(type='cpu')

In [2]:
import os
import random
import shutil

def split_dataset(source_dir, dest_dir, split_ratio=0.8):
    classes = os.listdir(source_dir)
    
    for cls in classes:
        cls_path = os.path.join(source_dir, cls)
        if not os.path.isdir(cls_path):
            continue

        images = [img for img in os.listdir(cls_path) if img.endswith(('.jpg', '.jpeg', '.png'))]
        random.shuffle(images)

        split_point = int(len(images) * split_ratio)
        train_imgs = images[:split_point]
        val_imgs = images[split_point:]

        for phase in ['train', 'val']:
            dest_cls_dir = os.path.join(dest_dir, phase, cls)
            os.makedirs(dest_cls_dir, exist_ok=True)

        # Copy images
        for img in train_imgs:
            shutil.copy(os.path.join(cls_path, img), os.path.join(dest_dir, 'train', cls, img))
        for img in val_imgs:
            shutil.copy(os.path.join(cls_path, img), os.path.join(dest_dir, 'val', cls, img))

    print("✅ Dataset split completed!")

# Usage
split_dataset('nigerian_attire', 'data_split', split_ratio=0.8)


✅ Dataset split completed!


### Step 3: Defining Augmentation and Transforms

In [3]:
from torchvision import transforms

train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.RandomAffine(degrees=15, translate=(0.05, 0.05)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])


### Step 4: Loading Data With PyTorch Dataloaders


In [4]:
# Load images from folders
train_data = datasets.ImageFolder('data_split/train', transform=train_transforms)
val_data = datasets.ImageFolder('data_split/val', transform=val_transforms)

# Load into batches
train_loader = DataLoader(train_data, batch_size=16, shuffle=True)
val_loader = DataLoader(val_data, batch_size=16, shuffle=False)

# Check classes
print("Classes:", train_data.classes)


Classes: ['fulani', 'hausa', 'igbo', 'yoruba']


### Step 5: Defining the Model


In [5]:
# Load a pre-trained ResNet18 model
model = models.resnet18(pretrained=True)

# Replace the final fully-connected layer to match your 4 attire classes
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 4)  # 4 output classes

model = model.to(device)  # Move model to GPU or CPU
model


/home/naziruai/miniconda3/envs/deep_learning/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/naziruai/miniconda3/envs/deep_learning/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

### Step 6: Defining the Loss Function and Optimizer

In [6]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


### Step 7: Training loop

In [7]:
# Train the model
num_epochs = 10
best_accuracy = 0.0

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Validation after each epoch
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss:.4f}, Val Accuracy: {accuracy:.2f}%")

    # Save best model
    if accuracy > best_accuracy:
        torch.save(model.state_dict(), 'best_model.pth')
        best_accuracy = accuracy


Epoch 1/10, Loss: 73.4673, Val Accuracy: 36.80%
Epoch 2/10, Loss: 61.1359, Val Accuracy: 44.98%
Epoch 3/10, Loss: 54.0195, Val Accuracy: 59.48%
Epoch 4/10, Loss: 48.0997, Val Accuracy: 59.11%
Epoch 5/10, Loss: 45.8159, Val Accuracy: 52.42%
Epoch 6/10, Loss: 47.6330, Val Accuracy: 60.22%
Epoch 7/10, Loss: 43.7606, Val Accuracy: 53.53%
Epoch 8/10, Loss: 41.7428, Val Accuracy: 57.99%
Epoch 9/10, Loss: 40.5209, Val Accuracy: 71.38%
Epoch 10/10, Loss: 36.5746, Val Accuracy: 61.71%


### Step 8: Finishing the training loop

In [8]:
print("✅ Training complete. Best validation accuracy: {:.2f}%".format(best_accuracy))


✅ Training complete. Best validation accuracy: 71.90%
